Updating melissa branch and main branch

Main notebook contains contributions from all members and is currently being reviewed for conformity.
Some order has been changed to ensure variables are set and core code is isolated outside of 
each person's contribution.

In [ ]:
#Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy.stats as st
from scipy.stats import linregress
import warnings
warnings.filterwarnings("ignore")

In [ ]:
measles_raw_data = pd.read_csv("measles - measles.csv")
measles_raw_data.head()
measles_raw_data.groupby("year").count()

In [ ]:
measles_clean = measles_raw_data.drop(columns=["xper", "xmed", "xrel", "district", "year", "county", "enroll"])
# measles_clean = measles_clean.dropna()

measles_clean.dtypes

In [ ]:
# neg_count = measles_clean.loc[["mmr" == "-1"]]
# len(neg_count)

measles_clean['mmr'].value_counts()
measles_clean['overall'].value_counts()

In [ ]:
# Dealing with the NaN's
measles_clean["type"].fillna("Other", inplace = True)

In [ ]:
# Remove -1 values
measles_clean = measles_clean.loc[measles_clean["mmr"] >= 0]
measles_clean = measles_clean.loc[measles_clean["overall"] >= 0]
measles_clean

In [ ]:
type_groups = measles_clean.groupby("type")
type_groups.count()

In [ ]:
# Terra's code begins here

In [ ]:
# Make series to create pie chart
school_type = measles_clean.groupby('type')

count_school_type = school_type['type'].count()
count_school_type

In [ ]:
# Fix error - Terra 
# No error after prior code order was corrected.  "Other" category had not been set.
school_type_pie = count_school_type.plot(kind="pie", figsize = (8,8), subplots=True, fontsize=15, autopct='%0.1f%%', \
                                         labels=['Kindergarten', 'Other', 'Private', 'Public'], title=['School Type']) 
# school_type_pie.set_ylabel("Types of Schools")
# plt.axis('equal') 
# plt.title('Type of School')

plt.show()                           
plt.tight_layout()
# type_vals = type_count['type']
# type_labels = type_count['index']

In [ ]:
# State list correct after prior code conformed and corrected
state_percents = measles_clean.groupby('state')

count_state_percents = state_percents['type'].count()
count_state_percents

In [ ]:
school_type_pie = count_state_percents.plot(kind="pie", figsize = (8,8), subplots=True, fontsize=14, autopct='%0.1f%%', \
                                         labels=['California', 'Colorado', 'Ohio', 'Oregon', 'Vermont', 'Washington'], \
                                         title=['Number of Schools by State']) 

# texts[0].set_fonsize(4)
plt.show()                           
plt.tight_layout()

In [ ]:
# Terra's code ends above
# Fukhrudin's code starts below

## SCATTER PLOT: LATITUDE vs LONGITUDE

In [ ]:
df1 = pd.DataFrame(measles_clean, columns=["lat", "lng"])
df1.plot.scatter(x="lng", y="lat");

plt.title("Scatter Plot: Latitude vs Longitude")
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.savefig('s1.png')

Points on the extreme right are erroneous and must be dropped from the table. The easternmost point in the contiguous US is West Quoddy Head Lighthouse (Maine) with the coordinates (44.81756628922163, -66.95132155560584) so we can't have any longitude value greater than - 66.95132155560584.

In [ ]:
# Keeping only negative longitudes
measles_clean_lng = measles_clean[measles_clean['lng'] < 0]

In [ ]:
# Re-drawing the scatter
df2 = pd.DataFrame(measles_clean_lng, columns=["lat", "lng"])
df2.plot.scatter(x="lng", y="lat");

plt.title("Scatter Plot: Latitude vs Longitude")
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.savefig('s2.png')

In [ ]:
# In re-establishing Fukhrudin's code, this cell is throwing a pink error
# Any need to worry? Or just carry on?

In [ ]:
#New column for coloring states
measles_clean_lng['state2'] = measles_clean_lng['state']

In [ ]:
# Assigning numerical values to state names for coloring
measles_clean_lng.loc[measles_clean_lng['state'] == 'California', 'state2'] = 1
measles_clean_lng.loc[measles_clean_lng['state'] == 'Colorado', 'state2'] = 2
measles_clean_lng.loc[measles_clean_lng['state'] == 'Ohio', 'state2'] = 3
measles_clean_lng.loc[measles_clean_lng['state'] == 'Oregon', 'state2'] = 4
measles_clean_lng.loc[measles_clean_lng['state'] == 'Washington', 'state2'] = 5
measles_clean_lng.loc[measles_clean_lng['state'] == 'Vermont', 'state2'] = 6

In [ ]:
# Colored scatte rplot
dfScat = measles_clean_lng[['lng','lat', 'state2']]
#axScat = dfScat.plot(kind='scatter', x=0, y=1, c=2, colormap='inferno')
#axScat = dfScat.plot(kind='scatter', x=0, y=1, c=2, colormap='plasma')
#axScat = dfScat.plot(kind='scatter', x=0, y=1, c=2, colormap='viridis')
axScat = dfScat.plot(kind='scatter', x=0, y=1, c=2, colormap='magma')

In [ ]:
# Boxplot of school types
# Looks like this is not Fukhrudin's code. Shoot.
type_boxplot = measles_clean_lng[['mmr','type']]
type_boxplot.boxplot(by='type')

In [ ]:
# Melissa's code starts here

In [ ]:
# Set up Public, Private, Kindergarten, and Other dataframes

public_stats = measles_clean.loc[measles_clean["type"] == "Public"]
# public_stats

private_stats = measles_clean.loc[measles_clean["type"] == "Private"]
# private_stats

kindergarten_stats = measles_clean.loc[measles_clean["type"] == "Kindergarten"]
# kindergarten_stats

other_stats = measles_clean.loc[measles_clean["type"] == "Other"]
# other_stats

In [ ]:
# Loop through the data frame to calculate stats by state

# Set up empty lists to fill in for loop
states = measles_clean["state"].unique()
# states
count_states = []
mean_mmr = []
median_mmr = []
public_mean = []
public_median = []
private_mean = []
private_median = []
kind_mean = []
kind_median = []
other_mean = []
other_median = []

for state in states:
    
    # Identify what state to work with
    state_active = measles_clean.loc[measles_clean["state"] == state]
    
    # Count all the schools for each state
    count_active = state_active["state"].count()
    count_states.append(count_active)
    
    # Calculate mean for each state
    mean_active = state_active["mmr"].mean()
    mean_mmr.append(mean_active)
    
    # Calculate median for each state
    median_active = state_active["mmr"].median()
    median_mmr.append(median_active)
    
    # Reset state_active for use on public subsets
    state_active = public_stats.loc[public_stats["state"] == state]
    
    # Calculate mean for public schools
    public_active = state_active["mmr"].mean()
    public_mean.append(public_active)
    
    # Calculate median for public schools
    public_active = state_active["mmr"].median()
    public_median.append(public_active)
    
    # Reset state_active for use on private subsets
    state_active = private_stats.loc[private_stats["state"] == state]
    
    # Calculate mean for private schools
    private_active = state_active["mmr"].mean()
    private_mean.append(private_active)
    
    # Calculate median for private schools
    private_active = state_active["mmr"].median()
    private_median.append(private_active)
    
    # Reset state_active for use on kindergarten subsets
    state_active = kindergarten_stats.loc[kindergarten_stats["state"] == state]
    
    # Calculate mean for kindergarten schools
    kind_active = state_active["mmr"].mean()
    kind_mean.append(kind_active)
    
    # Calculate median for kindergarten schools
    kind_active = state_active["mmr"].median()
    kind_median.append(kind_active)
    
    # Reset state_active for use on other subsets
    state_active = other_stats.loc[other_stats["state"] == state]
    
    # Calculate mean for Other schools
    other_active = state_active["mmr"].mean()
    other_mean.append(other_active)
    
    # Calculate median for Other schools
    other_active = state_active["mmr"].median()
    other_median.append(other_active)
   
       
# other_median

In [ ]:
# Set up summary table of averages per state
avg_by_state = pd.DataFrame({
    "State": states
})

avg_by_state["Count of Schools"] = count_states
avg_by_state["Overall Mean"] = mean_mmr
avg_by_state["Overall Median"] = median_mmr
avg_by_state["Public Mean"] = public_mean
avg_by_state["Public Median"] = public_median
avg_by_state["Private Mean"] = private_mean
avg_by_state["Private Median"] = private_median
avg_by_state["Kindergarten Mean"] = kind_mean
avg_by_state["Kindergarten Median"] = kind_median
avg_by_state["Other Mean"] = other_mean
avg_by_state["Other Median"] = other_median

avg_by_state

# Investigate NaNs
# Indicate a lack of that school type reporting out of that state

By setting up this summary table, we can see which states provided data for which types of schools.  This allows us to have an informed way of breaking the data apart and compare like to like data, such as California and Ohio, or Oregon, Vermon, and Washington.

In [ ]:
# Plot of school counts across states

y_axis = count_states
x_axis = np.arange(len(states))

plt.bar(x_axis, y_axis, color="green")
plt.ylabel("Number of Schools")
plt.xlabel("State")
plt.title("Count of Schools in States in Dataset", fontsize=15)

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, states, rotation="45", fontsize=12)

plt.show()
# Look at California alone

Looking at the count of schools across the states allows us to know if our sample sizes in each state are comparable.  While based on sheer numbers, this chart shows they are not, what we do not know is whether the proportion of schools that reported data out of the entire population is similar across states.  Either way, it is clear from the start difference in the count of schools from California that that state is worth looking at on its own as well.

In [ ]:
# Plot with grouped bars -- MEAN AND MEDIAN BY STATE
# https://matplotlib.org/3.3.4/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py
# Add rates to bars, if possible

x = np.arange(len(states))  
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, mean_mmr, width, label="Mean")
rects2 = ax.bar(x + width/2, median_mmr, width, label="Median")

plt.ylabel("Average of MMR Innoculation Rates")
plt.xlabel("State")
plt.title("Mean and Median of MMR Innoculation Rates across States in Dataset", fontsize=15)
plt.legend(loc="lower right")

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, states, rotation="45", fontsize=12)

plt.show()

By displaying the mean and median vaccination rates together, we can see not only how each rate differs across states but how consistently the mean is lower than the median rate for each state.  What this tells us is that there may be lower reported rates of vaccination at a sufficient number of schools in each state to pull the mean rate down.  Looking at the median as well is a reminder that that the actual reported rates may be higher than the mean only would lead a person to infer.  Because this difference holds true across states, it feels like an important trend to hold in mind when using either form of average.

In [ ]:
# Grouped bar chart of all averages across all states and school types
# Formatting is being disagreable

group_by_state = avg_by_state.drop(["Count of Schools"], axis=1)

state_group_fig = group_by_state.plot.bar(rot=45)

plt.ylabel("Average of MMR Innoculation Rates")
plt.xlabel("State")
plt.title("Mean and Median of MMR Innoculation Rates across States and School Types in Dataset", fontsize=15)
plt.legend(loc="center")

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, states, rotation="45", fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

plt.show()

In [ ]:
# Grouped bar chart with isolated means and medians
# Medians
median_group_fig = median_by_state.plot.bar(rot=45)

plt.ylabel("Median of MMR Innoculation Rates")
plt.xlabel("State")
plt.title("Median of MMR Innoculation Rates across States and School Types in Dataset", fontsize=15)
plt.legend(loc="center")

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, states, rotation="45", fontsize=12)

plt.show()

In [ ]:
# Alternate California Ohio Grouped Plot
# This might be better/more useful if the ticks were the school type and each had 

california_ohio = avg_by_state.loc[avg_by_state["Public Mean"] >= 0]
california_ohio = california_ohio.drop(columns=["Count of Schools", 
                                                "Kindergarten Mean", "Kindergarten Median",
                                                "Other Mean", "Other Median"])
cal_oh_labels = california_ohio["State"]
california_group_fig = california_ohio.plot.bar(rot=45)
x_axis = np.arange(len(cal_oh_labels))
plt.ylabel("Average of MMR Innoculation Rates")
plt.xlabel("State")
plt.title("Mean and Median of MMR Innoculation Rates in California and Ohio", fontsize=15)
plt.legend(loc="center")

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, cal_oh_labels, rotation="45", fontsize=12)

plt.show()

In [ ]:
california_ohio = california_ohio.reset_index(drop=True)
california_ohio = california_ohio.drop(columns=["State"])
ohio_cali = california_ohio.T
# ohio_cali.rename(columns={0: "California", 2: "Ohio"})
ohio_cali

In [ ]:
ohio_cali
oh_cal_labels = ohio_cali.index
oh_cal_group_fig = ohio_cali.plot.bar(rot=45)
x_axis = np.arange(len(oh_cal_labels))
plt.ylabel("Average of MMR Innoculation Rates")
plt.xlabel("School and Average Type")
plt.title("Mean and Median of MMR Innoculation Rates in California and Ohio", fontsize=15)
legend_text=plt.legend(loc="center")
legend_text.get_texts()[0].set_text("California")
legend_text.get_texts()[1].set_text("Ohio")
# Legend text issues resolved with help from Stack Overflor
# https://stackoverflow.com/questions/23037548/change-main-plot-legend-label-text

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, oh_cal_labels, rotation="45", fontsize=12)

plt.show()

In [ ]:
# Testing Average by type
avg_by_type = avg_by_state.drop(columns=["Count of Schools"])
avg_by_type = avg_by_type.drop(columns=["State"])
avg_by_type = avg_by_type.T
avg_by_type

In [ ]:
# Chart of Mean and Medians grouped together

avg_by_type
type_labels = avg_by_type.index
type_group_fig = avg_by_type.plot.bar(rot=45)
x_axis = np.arange(len(type_labels))
plt.ylabel("Average of MMR Innoculation Rates")
plt.xlabel("School and Average Type")
plt.title("Mean and Median of MMR Innoculation Rates in All States", fontsize=15)
legend_text=plt.legend(loc="center")
legend_text.get_texts()[0].set_text("California")
legend_text.get_texts()[1].set_text("Colorado")
legend_text.get_texts()[2].set_text("Ohio")
legend_text.get_texts()[3].set_text("Oregon")
legend_text.get_texts()[4].set_text("Vermont")
legend_text.get_texts()[5].set_text("Washington")

# Legend text issues resolved with help from Stack Overflor
# https://stackoverflow.com/questions/23037548/change-main-plot-legend-label-text

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, type_labels, rotation="45", fontsize=12)

plt.show()

In [ ]:
print(states)
avg_by_state
mean_by_type = avg_by_state.drop(columns=["Count of Schools", "State", 
                                          "Overall Median", "Public Median", 
                                          "Kindergarten Median", "Other Median"])
mean_by_type = mean_by_type.T
mean_by_type

median_by_type = avg_by_state.drop(columns=["Count of Schools", "State", 
                                          "Overall Mean", "Public Mean", 
                                          "Kindergarten Mean", "Other Mean"])
median_by_type = median_by_type.T
median_by_type

In [ ]:
# Chart of Mean by type

mean_by_type
type_labels = mean_by_type.index
type_group_fig = mean_by_type.plot.bar(rot=45)
x_axis = np.arange(len(type_labels))
plt.ylabel("Average of MMR Innoculation Rates")
plt.xlabel("School and Average Type")
plt.title("Mean of MMR Innoculation Rates in All States", fontsize=15)
legend_text=plt.legend(loc="center")
legend_text.get_texts()[0].set_text("California")
legend_text.get_texts()[1].set_text("Colorado")
legend_text.get_texts()[2].set_text("Ohio")
legend_text.get_texts()[3].set_text("Oregon")
legend_text.get_texts()[4].set_text("Vermont")
legend_text.get_texts()[5].set_text("Washington")

# Legend text issues resolved with help from Stack Overflor
# https://stackoverflow.com/questions/23037548/change-main-plot-legend-label-text

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, type_labels, rotation="45", fontsize=12)

plt.show()

In [ ]:
# Chart for Median only

median_by_type
type_labels = median_by_type.index
type_group_fig = median_by_type.plot.bar(rot=45)
x_axis = np.arange(len(type_labels))
plt.ylabel("Average of MMR Innoculation Rates")
plt.xlabel("School and Average Type")
plt.title("Median of MMR Innoculation Rates in All States", fontsize=15)
legend_text=plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
legend_text.get_texts()[0].set_text("California")
legend_text.get_texts()[1].set_text("Colorado")
legend_text.get_texts()[2].set_text("Ohio")
legend_text.get_texts()[3].set_text("Oregon")
legend_text.get_texts()[4].set_text("Vermont")
legend_text.get_texts()[5].set_text("Washington")

# Legend text issues resolved with help from Stack Overflor
# https://stackoverflow.com/questions/23037548/change-main-plot-legend-label-text

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, type_labels, rotation="60", fontsize=12)


plt.show()

In [ ]:
# Grouped bar chart with isolated means and medians
# Means
mean_group_fig = mean_by_state.plot.bar(rot=45)

plt.ylabel("Mean of MMR Innoculation Rates")
plt.xlabel("State")
plt.title("Mean of MMR Innoculation Rates across States and School Types in Dataset", fontsize=15)
plt.legend(loc="center")

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, states, rotation="45", fontsize=12)

plt.show()

In [ ]:
# Melissa's code ends above
# Rylee's code begins here!

In [ ]:
# # Borrowed Melissa's variables in order to pull out values based on type
# # Set up Public, Private, Kindergarten, and Other dataframes

# public_stats = measles_clean.loc[measles_clean["type"] == "Public"]
# # public_stats

# private_stats = measles_clean.loc[measles_clean["type"] == "Private"]
# # private_stats

# kindergarten_stats = measles_clean.loc[measles_clean["type"] == "Kindergarten"]
# # kindergarten_stats

# other_stats = measles_clean.loc[measles_clean["type"] == "Other"]
# # other_stats

In [ ]:
# # Type averages plot setup

# public_mmr_mean = public_stats['mmr'].mean()
# # public_mmr_mean

# private_mmr_mean = private_stats['mmr'].mean()
# # private_mmr_mean

# kindergarten_mmr_mean = kindergarten_stats['mmr'].mean()
# # kindergarten_mmr_mean

# other_mmr_mean = other_stats['mmr'].mean()
# # other_mmr_mean

In [ ]:
# Just doing hypothesis testing right now
# We are trying to prove that mmr rates vary by school type

st.f_oneway(public_stats['mmr'], private_stats['mmr'], kindergarten_stats['mmr'], other_stats['mmr'])

ANOVA test for all school types in every state in clean dataset. NULL hypothesis is that the mean and variance in all school types are the same. The extremely small p-value indicates that these datasets are not alike, disproving the NULL. Further analysis needed to determine if one outlier is effecting the data or if there are differences between all that are significant.

Will repeat test after dropping outliers as well.

In [ ]:
# Taking out the "other" school type
st.f_oneway(public_stats['mmr'], private_stats['mmr'], kindergarten_stats['mmr'])

ANOVA test removing "Other" school type with the same NULL hypothesis (all types are the same), returns a very small p-value as well.  This value is not as small, indicating that this type had an impact on the p-value for all types together, but the remaining types still significantly different.

In [ ]:
# Taking out the "kindergarten" school type
st.f_oneway(public_stats['mmr'], private_stats['mmr'], other_stats['mmr'])

ANOVA test removing "Kindergarten" school type with the same NULL hypothesis (all types are the same), returns a very small p-value as well.  This value is even smaller than the overall p-value, indicating that this type had an impact on the p-value for all types together, but the remaining types still significantly different.

In [ ]:
# Taking out the "private" school type
st.f_oneway(public_stats['mmr'], kindergarten_stats['mmr'], other_stats['mmr'])

ANOVA test removing "Private" school type with the same NULL hypothesis (all types are the same), returns a very small p-value as well.  This value is very similar to the p-value overall, indicating that this type had a small impact on the p-value for all types together, but the remaining types still significantly different.

In [ ]:
# Taking out the "public" school type
st.f_oneway(private_stats['mmr'], kindergarten_stats['mmr'], other_stats['mmr'])

ANOVA test removing "Public" school type with the same NULL hypothesis (all types are the same), returns a very small p-value as well.  This value is the largest of the p-values, indicating the other three school types are more similar than any other combination of school types. This tells us that the Public school data differs from the other datasets more.

In [ ]:
# Taking out "other" and "public" and running a t-test because this is where the p-vals were the largest
# when I took them out in the ANOVA
st.ttest_ind(private_stats['mmr'], kindergarten_stats['mmr'], equal_var=False)

Comparing Private and Kindergarten datasets with a Two Sample T-Test with the NULL hypothesis that the means of both datasets are the same. With a p-value less than 0.05, we can reject the NULL and conclude that the means are different.

In [ ]:
# T-test on "other" and "private"
st.ttest_ind(private_stats['mmr'], other_stats['mmr'], equal_var=False)

Comparing Private and Other types with a Two Sample T-Test with the NULL hypothesis that the means of both datasets are the same. With a p-value less than 0.05, we can reject the NULL and conclude that the means are different.

In [ ]:
# T-test on "other" and "kindergarten"
st.ttest_ind(other_stats['mmr'], kindergarten_stats['mmr'], equal_var=False)

Comparing Other and Kindergarten with the NULL hypothesis that the means of both datasets are the same. With a p-value less than 0.05, we can reject the NULL and conclude that the means are different.

In [ ]:
# T-test on "public" and "private"
st.ttest_ind(private_stats['mmr'], public_stats['mmr'], equal_var=False)

Comparing Public and Private with the NULL hypothesis that the means of both datasets are the same. With a p-value less than 0.05, we can reject the NULL and conclude that the means are different.

In [ ]:
# T-test on "public" and "other"
st.ttest_ind(other_stats['mmr'], public_stats['mmr'], equal_var=False)

Comparing Public and Other with the NULL hypothesis that the means of both datasets are the same. With a p-value less than 0.05, we can reject the NULL and conclude that the means are different.

In [ ]:
# Other pairings as determined

In [ ]:
# Rylee's code ends here!